In [80]:
import math
class Value:
    def __init__(self,data,children=(),op='', label = '') -> None:
        self.data = data
        self._prev = set(children)
        self._op=op
        self.label = label
        self.grad = 0
        self._backward = lambda: None


    def __repr__(self) -> str:
        return f"Value: {self.data}"

    def __add__(self,other):
        other = other if isinstance(other,Value) else Value(other)
        out = Value(self.data+other.data,(self,other),'+')

        def _backward():
            self.grad += 1*out.grad
            other.grad += 1*out.grad
        
        out._backward = _backward
        return out
    
    def __mul__(self,other):
        other = other if isinstance(other,Value) else Value(other)
        out = Value(self.data*other.data,(self,other),'*')

        def _backward():
            self.grad += other.data*out.grad
            other.grad += self.data*out.grad
        
        out._backward = _backward

        return out
    
    def __neg__(self):
        return self*-1
    
    def __sub__(self,other):
        return self+(-other)

    def __rmul__(self,other):
        return self*other
    
    def __truediv__(self,other):
        return self*(other**-1)
    
    def __pow__(self,other):
        assert isinstance(other,(int,float))
        out = Value(self.data**other,(self,),'**{}'.format(other))


        def _backward():
            self.grad += other*(self.data**(other-1))*out.grad
        
        out._backward = _backward
        return out

    def tanh(self):
        x = self.data   
        t = (math.exp(2*x)-1)/(math.exp(2*x)+1)
        out = Value(t,(self,), 'tanh')

        def _backward():
            self.grad += (1-t**2)*out.grad
        
        out._backward = _backward
        return out
    
    def exp(self):
        x = self.data   
        t = math.exp(x)
        out = Value(t,(self,), 'tanh')

        def _backward():
            self.grad +=t*out.grad
        
        out._backward = _backward
        return out

    
    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        self.grad = 1
        for v in reversed(topo):
            v._backward()
    
a = Value(2)
b = Value(3)

(a*b)._prev


{Value: 2, Value: 3}

In [108]:
import random
class Neuron:
    def __init__(self,nin) -> None:
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1,1))
    
    def __call__(self,x)  :
        act = sum([wi*xi for wi,xi in zip(self.w,x)],self.b)
        out = act.tanh()
        return out
    
class Layer:
    def __init__(self,nin,nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]
    
    def __call__(self,x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs)==1 else outs
class MLP:
    def __init__(self,nin,nouts) -> None:
        sz = [nin]+nouts
        self.layers= [Layer(sz[i],sz[i+1]) for i in range(len(nouts))]
    
    def __call__(self,x):
        for layer in self.layers:
            x = layer(x)
        return x
    
x = [2,3,-1]
n = MLP(3,[4,4,1])
n(x)

[Value: 0.6462957384312107]